## 1. Data Exploration

Our sales department is interested in a summary of the collected data. Please generate a report including numbers and diagrams. Note that your audience are not data scientists, so take care to prepare insights that are as clear as possible. We are interested in the following:

1. Calculate the total number of customers in each section

2. Calculate the total number of customers in each section over time

3. Display the number of customers at checkout over time

4. Calculate the time each customer spent in the market

5. Calculate the total number of customers in the supermarket over time

6. Our business managers think that the first section customers visit follows a different pattern than the following ones. Plot the distribution of customers of their first visited section versus following sections (treat all sections visited after the first as “following”).

In [1]:
import pandas as pd
import datetime
from datetime import timedelta

In [2]:
# Reading the data
monday = pd.read_csv('../data/monday.csv', sep=';')
tuesday = pd.read_csv('../data/tuesday.csv', sep=';')
wednesday =  pd.read_csv('../data/wednesday.csv', sep=';')
thursday = pd.read_csv('../data/thursday.csv', sep=';')
friday = pd.read_csv('../data/friday.csv', sep=';')

In [3]:
monday.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4884 entries, 0 to 4883
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   timestamp    4884 non-null   object
 1   customer_no  4884 non-null   int64 
 2   location     4884 non-null   object
dtypes: int64(1), object(2)
memory usage: 114.6+ KB


In [4]:
monday.describe()

,customer_no
count,4884.000000
mean,718.274365
std,411.839636
min,1.000000
25%,366.000000
50%,720.000000
75%,1070.000000
max,1447.000000


In [5]:
monday['timestamp'] = pd.to_datetime(monday['timestamp'])
tuesday['timestamp'] = pd.to_datetime(tuesday['timestamp'])
wednesday['timestamp'] = pd.to_datetime(wednesday['timestamp'])
thursday['timestamp'] = pd.to_datetime(thursday['timestamp'])
friday['timestamp'] = pd.to_datetime(friday['timestamp'])

In [6]:
# Сreating new columns:
# Monday
monday['time'] = monday['timestamp'].dt.time
monday['minutes'] = monday['timestamp'].dt.minute

#tuesday['time']=monday['timestamp'].dt.time
#wednesday['time']=monday['timestamp'].dt.time
#thursday['time']=monday['timestamp'].dt.time
#friday['time']=monday['timestamp'].dt.time

monday

,timestamp,customer_no,location,time,minutes
0,2019-09-02 07:03:00,1,dairy,07:03:00,3
1,2019-09-02 07:03:00,2,dairy,07:03:00,3
2,2019-09-02 07:04:00,3,dairy,07:04:00,4
3,2019-09-02 07:04:00,4,dairy,07:04:00,4
4,2019-09-02 07:04:00,5,spices,07:04:00,4
...,...,...,...,...,...
4879,2019-09-02 21:49:00,1442,checkout,21:49:00,49
4880,2019-09-02 21:49:00,1444,checkout,21:49:00,49
4881,2019-09-02 21:49:00,1445,dairy,21:49:00,49
4882,2019-09-02 21:50:00,1446,dairy,21:50:00,50


In [7]:
# Creating new column in each data set:
monday['weekday'] = 'monday'
tuesday['weekday'] = 'tuesday'
wednesday['weekday'] = 'weekday'
thursday['weekday'] = 'thursday'
friday['weekday'] = 'friday'

# Combining all datasets together:
days = [monday, tuesday, wednesday, thursday, friday]
weekday = pd.concat(days)
weekday;

### 1. Calculate the total number of customers in each section

In [8]:
monday_customers = monday.groupby('location').nunique()
tuesday_customers = tuesday.groupby('location').nunique()
wednesday_customers = wednesday.groupby('location').nunique()
thursday_customers = thursday.groupby('location').nunique()
friday_customers = friday.groupby('location').nunique()

print(monday_customers.customer_no)
print(tuesday_customers.customer_no)
print(wednesday_customers.customer_no)
print(thursday_customers.customer_no)
print(friday_customers.customer_no)

location
checkout    1437
dairy        720
drinks       661
fruit        827
spices       584
Name: customer_no, dtype: int64
location
checkout    1420
dairy        751
drinks       581
fruit        827
spices       543
Name: customer_no, dtype: int64
location
checkout    1526
dairy        804
drinks       652
fruit        884
spices       565
Name: customer_no, dtype: int64
location
checkout    1532
dairy        782
drinks       632
fruit        872
spices       613
Name: customer_no, dtype: int64
location
checkout    1502
dairy        761
drinks       688
fruit        874
spices       633
Name: customer_no, dtype: int64


### 2. Calculate the total number of customers in each section over time

In [9]:
total = monday_customers + tuesday_customers + wednesday_customers + thursday_customers + friday_customers
total[['customer_no']]

,customer_no
location,
checkout,7417
dairy,3818
drinks,3214
fruit,4284
spices,2938


### 3. Display the number of customers at checkout over time

In [10]:
total_checkout = total.loc[['checkout']][['customer_no']]
total_checkout

,customer_no
location,
checkout,7417


### 4. Calculate the time each customer spent in the market

In [11]:
max_time_monday = monday.groupby("customer_no")["timestamp"].max()
min_time_monday = monday.groupby("customer_no")["timestamp"].min()

time_spent = pd.concat([min_time_monday, max_time_monday], axis=1)
time_spent['time_spent'] = (max_time_monday - min_time_monday)
time_spent[['time_spent']]

,time_spent
customer_no,
1,0 days 00:02:00
2,0 days 00:03:00
3,0 days 00:02:00
4,0 days 00:04:00
5,0 days 00:01:00
...,...
1443,0 days 00:01:00
1444,0 days 00:01:00
1445,0 days 00:00:00


### 5. Calculate the total number of customers in the supermarket over time

In [12]:
total_supermarket = monday_customers.max() + tuesday_customers.max() + wednesday_customers.max() + thursday_customers.max() + friday_customers.max()
print('Total number of customers in the supermarket:',total_supermarket['customer_no'])

Total number of customers in the supermarket: 7417.0


### 6. Plot the distribution of customers of their first visited section versus following sections (Transitions)

### 7. Revenue

In [13]:
# Creating data frame:
revenue = pd.DataFrame({'location': ['fruit', 'spices', 'dairy', 'drinks'], 'revenue_per_minute': [4, 3, 5, 6]})
revenue

,location,revenue_per_minute
0,fruit,4
1,spices,3
2,dairy,5
3,drinks,6


In [14]:
# Setting values to the column 'revenue' by location:
monday.loc[monday["location"] == "fruit", "revenue"] = "4"
monday.loc[monday["location"] == "spices", "revenue"] = "3"
monday.loc[monday["location"] == "dairy", "revenue"] = "5"
monday.loc[monday["location"] == "drinks", "revenue"] = "6"
monday.loc[monday["location"] == "checkout", "revenue"] = "0"

#monday[['customer_no', 'location', 'revenue']]
monday

,timestamp,customer_no,location,time,minutes,weekday,revenue
0,2019-09-02 07:03:00,1,dairy,07:03:00,3,monday,5
1,2019-09-02 07:03:00,2,dairy,07:03:00,3,monday,5
2,2019-09-02 07:04:00,3,dairy,07:04:00,4,monday,5
3,2019-09-02 07:04:00,4,dairy,07:04:00,4,monday,5
4,2019-09-02 07:04:00,5,spices,07:04:00,4,monday,3
...,...,...,...,...,...,...,...
4879,2019-09-02 21:49:00,1442,checkout,21:49:00,49,monday,0
4880,2019-09-02 21:49:00,1444,checkout,21:49:00,49,monday,0
4881,2019-09-02 21:49:00,1445,dairy,21:49:00,49,monday,5
4882,2019-09-02 21:50:00,1446,dairy,21:50:00,50,monday,5


In [15]:
# monday.sort_values(by=['location', 'revenue']).head()

In [16]:
# Converting revenue column into integer: 
monday['revenue'] = monday['revenue'].astype(int)

In [17]:
# Calculatting revenue by location:
monday.groupby('location')[['revenue']].sum().sort_values('revenue', ascending=False)

,revenue
location,
drinks,4782
dairy,4475
fruit,4020
spices,2250
checkout,0


In [18]:
# Calculatting revenue from every customer:
revenue_per_customer = monday.groupby('customer_no')[['revenue']].sum().sort_values('revenue', ascending=False)
revenue_per_customer

,revenue
customer_no,
526,72
380,64
666,56
391,50
86,49
...,...
551,3
1196,3
559,3


In [19]:
# Calculating total revenue per day (monday):
revenue_per_customer.sum()

revenue    15527
dtype: int64

In [39]:
# New strategy to calculate the revenue based on the time spent in the market

def revenue_per_customer(df, customer_no):
    """
    Given a dataframe creates new dataframes
    with the revenue made per customer per day
    """

    for i in range(df["customer_no"].max()):
        customer_data = monday[monday["customer_no"].isin([customer_no])].copy()
        customer_data['location_before'] = customer_data['location'].shift(1).fillna('entrance')
        customer_data['timnestamp_before'] = customer_data['timestamp'].shift(1)
        customer_data['time_spent'] = customer_data['timestamp'] - customer_data['timnestamp_before']
        customer_data['time_spent'] = customer_data['time_spent'].shift(-1)
        return customer_data

df1 = revenue_per_customer(monday, 6)
df1

,timestamp,customer_no,location,time,minutes,weekday,revenue,location_before,timnestamp_before,time_spent
5,2019-09-02 07:04:00,6,spices,07:04:00,4,monday,3,entrance,NaT,0 days 00:01:00
10,2019-09-02 07:05:00,6,dairy,07:05:00,5,monday,5,spices,2019-09-02 07:04:00,0 days 00:12:00
62,2019-09-02 07:17:00,6,spices,07:17:00,17,monday,3,dairy,2019-09-02 07:05:00,0 days 00:01:00
71,2019-09-02 07:18:00,6,drinks,07:18:00,18,monday,6,spices,2019-09-02 07:17:00,0 days 00:07:00
102,2019-09-02 07:25:00,6,spices,07:25:00,25,monday,3,drinks,2019-09-02 07:18:00,0 days 00:01:00
104,2019-09-02 07:26:00,6,dairy,07:26:00,26,monday,5,spices,2019-09-02 07:25:00,0 days 00:01:00
117,2019-09-02 07:27:00,6,spices,07:27:00,27,monday,3,dairy,2019-09-02 07:26:00,0 days 00:02:00
135,2019-09-02 07:29:00,6,checkout,07:29:00,29,monday,0,spices,2019-09-02 07:27:00,NaT
